<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Actuals Specifications </span> </p>

The aim of this notebook is to document the calculations taking place when importing the Actuals. For this documentation notebook the Data is imported from the corresponding csv files located in the following path <code>Initialization/Systemorph/</code>. The process is pictorially explained in the following flowchart. 

<a name="TOC"></a>
<center><img src="../Images/SpecificationsImportActuals.PNG" width="45%" style="float:center;">

Throughout this notebook we adopt the following variable notation:  we use the words *computed* vs. *expected* to differentiate the results provided by the application vs. this test notebook, respectively.

# Import Data

<a id='year-and-period'></a>
## Current Period

Please select the target period (i.e. year and month), Reporting Node, Economic Basis, and Data Node based on the imported data:

In [ ]:
var year          = 2021   ;
var month         = 3      ;
var reportingNode = "CH"   ;
var economicBasis = "L"    ;
var dataNode      = "DT1.1";

Import the Dimensions, Yield Curves and Nominal Cashflows from the corresponding csv files in <code>Initialization/Systemorph/</code>

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

---

[Setup IFRS17 Calculation Engine](../Import/ImportScopeCalculation):

In [ ]:
#!import "SpecificationsSetup"

---

Initialization of the Test Suite: the data loaded from csv files are ready to be used by the Scopes for the calculations

In [ ]:
var Test = await StartCalculatorAsync(reportingNode, year, month, economicBasis, Periodicity.Quarterly, ImportFormats.Actual);

<a id='view-imported-actuals'></a>
## View imported Actuals

Here the imported Actuals from csv are visualized 

In [ ]:
Test.GetStorage().GetIfrsVariables(dataNode)
    .Where(x => new string[] {EstimateTypes.A, EstimateTypes.AA, EstimateTypes.OA}.Contains(x.EstimateType))
    .Where(x => x.AocType != "EOP")

where the filter is selecting only those *estimate types* associated to the imported Actuals, hence excluding the IFRS variables calculated when [importing the cashflows](SpecificationsImportCashflows).

<br/><br/>
<a id='base-actual'></a>
# Base Actuals

The following simplified AoC Chain applies for Advance and Overdue Actuals:
1. Beginning of Period (BOP)
2. Cash flow (CF)
3. Amortization (AM) 
4. End of Period (EOP) 

We consider 4 types of Actual values, which are distinguished through their [Estimate Type](../DataModel/DataStructure#estimate-type):
- Actuals (A)
- Advance Actuals (AA)
- Overdue Actuals (OA)
- Deferrable Actuals (DA)

with the Estimate Type's system name shown between parenthesis above.
They are computed by the so-called Actual Base **Scope**

$$
\text{Actual Base} (\text{AoC}) = \left\{
\begin{array}{cl}
0                                                                                        & \text{if AoC step is AM,} \\
\text{Actual Base}(\rm{BOP}) + \text{Actual Base}(\rm{CF}) + \text{Actual Base}(\rm{WO}) & \text{if AoC step is EOP and Estimate Type is not A,} \\
\text{Imported Actual}                                                                   & \text{otherwise.}
\end{array}
\right.
$$

We start by selecting one of the [imported Actual](#view-imported-actuals) and we pick up an available data node, novelty, and amount type. Then, we can retrieve the corresponding actual (if any) with AoC type BOP, R, and WO. 

Note that the **Identity** for Actual Base contains information about the target data node, AoC type, novelty, amount type and estimate type.

## End of Period

The imported actuals for the AoC steps BOP, CF, and WO are:

In [ ]:
var actualBaseBOP = Test.GetScope<ActualBase>( (new ImportIdentity {DataNode = "DT1.1", AocType = "BOP", Novelty = "I" }, "PR", "AA", (int?)null) ); 
actualBaseBOP

In [ ]:
var actualBaseCF = Test.GetScope<ActualBase>( (new ImportIdentity {DataNode = "DT1.1", AocType = "CF", Novelty = "C" }, "PR", "AA", (int?)null) );
actualBaseCF

In [ ]:
var actualBaseWO = Test.GetScope<ActualBase>( (new ImportIdentity {DataNode = "DT1.1", AocType = "WO", Novelty = "C" }, "PR", "AA", (int?)null) ); 
actualBaseWO

Based on the imported actuals and according to the [formula above](#base-actual), the computed actual for AoC step EOP reads

In [ ]:
var computedActualBaseEOP = Test.GetScope<ActualBase>( (new ImportIdentity {DataNode = "DT1.1", AocType = "EOP", Novelty = "C" }, "PR", "AA", (int?)null) ); 
computedActualBaseEOP

The computed value can be compared with the expected value

In [ ]:
computedActualBaseEOP.Value == actualBaseBOP.Value + actualBaseCF.Value + actualBaseWO.Value

In [ ]:
computedActualBaseEOP.Value.Should().Be( actualBaseBOP.Value + actualBaseCF.Value + actualBaseWO.Value );

<br/><br/>
<a id='deferrable-actual'></a>
# Deferrable Actuals

The Deferrable Actuals (DA) are computed as follows

$$
\text{Deferrable Actual} (\text{AoC}) = \left\{
\begin{array}{cl}
\text{Actual Base}_{\substack{\text{Estimate Type}=\text{A} \\ \text{Amount Type}=\text{ACA}}} (\text{CF})
 +\text{Actual Base}_{\substack{\text{Estimate Type}=\text{A} \\ \text{Amount Type}=\text{AEA}}} (\text{CF})    & \text{if AoC step is CF }  \\
-\text{AMF} \cdot \big( \text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) \big)            & \text{if AoC step is AM } \\
\text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) + \text{Deferrable Actual}(\rm{AM})      & \text{if AoC step is EOP } \\
\text{Input Actual}_{\text{Estimate Type}=\text{DA}} (\text{BOP})                                              & \text{if AoC step is BOP } \\
\end{array}
\right.
$$

where ACA and AEA are *Aquisition* Amount Types from *Attributable Commission* and *Attributable Expenses*, respectively.

Note that the **Identity** of Deferrable Actual is encompassing solely data node, Aoc type, and novelty.

## Beginning of Period

In [ ]:
var computedDeferrableActualBOP = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "BOP", Novelty = "I" } );
computedDeferrableActualBOP

The deferrable Actual for the AoC step being BOP is simply the EOP Actual of the previous period. In fact, if the latter exists among the [imported Actuals](#view-imported-actuals), it is reported here below as the BOP of the current period:

In [ ]:
var expectedDeferrableActualBOP = Test.GetStorage().GetValue(new ImportIdentity {DataNode = "DT1.1", AocType = "BOP", Novelty = "I" }, (string)null, EstimateTypes.DA, (int?)null);
expectedDeferrableActualBOP

The expected value is compared with the computed result: 

In [ ]:
computedDeferrableActualBOP.Value == expectedDeferrableActualBOP

In [ ]:
computedDeferrableActualBOP.Value.Should().Be(expectedDeferrableActualBOP);

## Cash flow

In [ ]:
var computedDeferrableActualCF = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "CF", Novelty = "C" } );
computedDeferrableActualCF

The deferrable Actual for the AoC step being CF is the sum of the base Actuals with Amount Type ACA and AEA:

In [ ]:
var actualBaseAEA = Test.GetScope<ActualBase>( (new ImportIdentity {DataNode = "DT1.1", AocType = "CF", Novelty = "C" }, "AEA", "A", (int?)null) );
actualBaseAEA

In [ ]:
var actualBaseACA = Test.GetScope<ActualBase>( (new ImportIdentity {DataNode = "DT1.1", AocType = "CF", Novelty = "C" }, "ACA", "A", (int?)null) );
actualBaseACA

In [ ]:
var expectedDeferrableActualCF = actualBaseAEA.Value + actualBaseACA.Value;
expectedDeferrableActualCF

The expected value is compared with the computed result: 

In [ ]:
computedDeferrableActualCF.Value == expectedDeferrableActualCF

In [ ]:
computedDeferrableActualCF.Value.Should().Be(expectedDeferrableActualCF);

## Amortization

In [ ]:
var computedDeferrableActualAM = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "AM", Novelty = "C" } );
computedDeferrableActualAM

The deferrable Actual for the AoC step being AM is the sum of the deferrable Actuals for AoC step BOP and CF multiplied by the current period amortization factor:

In [ ]:
var currentPeriodAmortizationFactor = Test.GetScope<CurrentPeriodAmortizationFactor>( new ImportIdentity {DataNode = "DT1.1", AocType = "AM", Novelty = "C" } );
currentPeriodAmortizationFactor

In [ ]:
Test.GetStorage().GetValue(new ImportIdentity {DataNode = "DT1.1", AocType = "AM", Novelty = "C" }, (string)null, "F", "L", (int?)null)

In [ ]:
var expectedDeferrableActualAM = -currentPeriodAmortizationFactor.Value * (expectedDeferrableActualBOP + expectedDeferrableActualCF);
expectedDeferrableActualAM

The expected value is compared with the computed result: 

In [ ]:
computedDeferrableActualAM.Value == expectedDeferrableActualAM

In [ ]:
computedDeferrableActualAM.Value.Should().Be( expectedDeferrableActualAM );

## End of Period

In [ ]:
var computedDeferrableActualEOP = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "EOP", Novelty = "C" } );
computedDeferrableActualEOP

The deferrable Actual for the AoC step being EOP is the sum of the deferrable Actuals for AoC step BOP, CF, and AM:

In [ ]:
var deferrableActualBOP = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "BOP", Novelty = "I" } );
deferrableActualBOP

In [ ]:
var deferrableActualCF = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "CF", Novelty = "C" } );
deferrableActualCF

In [ ]:
var deferrableActualAM = Test.GetScope<DeferrableActual>( new ImportIdentity {DataNode = "DT1.1", AocType = "AM", Novelty = "C" } );
deferrableActualAM

In [ ]:
var expectedDeferrableActualEOP = deferrableActualBOP.Value + deferrableActualCF.Value + deferrableActualAM.Value;
expectedDeferrableActualEOP

The expected value is compared with the computed result: 

In [ ]:
computedDeferrableActualEOP.Value == expectedDeferrableActualEOP

In [ ]:
computedDeferrableActualEOP.Value.Should().Be(expectedDeferrableActualEOP);